In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp

spark = SparkSession.builder \
    .appName("Coin Price Streaming") \
    .getOrCreate()

df_static = spark.read.format("parquet").load("E:/Spark_Kafka_Coin/data-coin")
schema = df_static.schema

df_stream = (
    spark.readStream
    .format("parquet")
    .schema(schema)  
    .load("E:/Spark_Kafka_Coin/data-coin")
)

df_stream = df_stream.withColumn("timestamp", to_timestamp(col("timestamp"), "yyyy-MM-dd HH:mm:ss"))

df_stream = df_stream.select("timestamp", "price")


In [3]:

query = df_stream.writeStream \
    .format("memory") \
    .queryName("coin_prices") \
    .outputMode("append") \
    .start()

In [14]:
from time import sleep
from IPython.display import clear_output
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.animation as animation


def read_data():
        coin_df = spark.sql("SELECT * FROM coin_prices")
        return coin_df.toPandas()

while True:    
    clear_output(wait=True)
    df = read_data()
    if not df.empty:
        plt.clf()
        plt.plot(df['timestamp'], df['price'], label='Price')
        plt.xlabel('Timestamp')
        plt.ylabel('Price')
        plt.title('Coin Price Over Time')
        plt.legend()

    fig = plt.figure()
    plt.show()
    sleep(10)

TypeError: Value after * must be an iterable, not int